In [1]:
debug = False

___
___
# **Librairies**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.model_selection import KFold, StratifiedKFold, RepeatedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.ensemble import VotingRegressor

import xgboost as xgb
import lightgbm as lgb
import catboost
from lightgbm.sklearn import LGBMClassifier
from catboost import CatBoostClassifier
from tqdm import tqdm

from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


___
___
# **Open Data**

In [3]:
df       = pd.read_csv('/kaggle/input/trustii-processed-data/train.csv')
test_df  = pd.read_csv('/kaggle/input/trustii-processed-data/test.csv')

if 'trustii_id' in df :
    df = df.drop(columns = ['trustii_id'])

print(df.shape, test_df.shape)
display(df.head())

(22671, 921) (9717, 922)


,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone,Ville,Departement,flag_Abattage de produits de l'aquaculture,flag_Alimentation générale,flag_Autres activités de remise directe,flag_Boucherie-Charcuterie,flag_Boulangerie-Pâtisserie,flag_Caves d'affinage,flag_Centre d'emballage des oeufs,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Chaîne d'abattage de volaille/lagomorphe/petit gibier,flag_Chocolatier,flag_Collecte de gibier sauvage,flag_Collecte de lait,flag_Collecte et transformation de miel,flag_Collecteur d'oeufs,flag_Commerce alimentaire,flag_Distribution automatique,flag_Découpe de fromages,flag_Découpe de gibier sauvage,flag_Découpe de gros gibier d'élevage,flag_Découpe de produits de charcuterie,flag_Découpe de viande de boucherie,flag_Découpe de viandes de volailles/lagomorphe,flag_Déshydratation de lait ou produits laitiers,flag

In [4]:
dico_y = {'A corriger de manière urgente' : 0,
          'A améliorer' : 1,
          'Satisfaisant' : 2,
          'Très satisfaisant' : 3,
         }
dico_y_INVERSE = {v:k for k, v in dico_y.items()}

In [5]:
df['y'] = df['Synthese_eval_sanit'].map(dico_y)

___
___
# **Modélisation**

In [7]:
df = df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

df.head(2)

,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite,APP_Libelle_activite_etablissement_initial,Libelle_commune_initial,APP_Libelle_etablissement_initial,Adresse_2_UA_initial,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone,Ville,Departement,flag_Abattage de produits de l'aquaculture,flag_Alimentation générale,flag_Autres activités de remise directe,flag_Boucherie-Charcuterie,flag_Boulangerie-Pâtisserie,flag_Caves d'affinage,flag_Centre d'emballage des oeufs,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Chaîne d'abattage de volaille/lagomorphe/petit gibier,flag_Chocolatier,flag_Collecte de gibier sauvage,flag_Collecte de lait,flag_Collecte et transformation de miel,flag_Collecteur d'oeufs,flag_Commerce alimentaire,flag_Distribution automatique,flag_Découpe de fromages,flag_Découpe de gibier sauvage,flag_Découpe de gros gibier d'élevage,flag_Découpe de produits de charcuterie,flag_Découpe de viande de boucherie,flag_Découpe de viandes de volailles/lagomorphe,flag_Déshydratation de lait ou produits laitiers,flag

In [8]:
useless = []

forbidden = ['APP_Libelle_etablissement',
             'APP_Libelle_etablissement_initial',
             'APP_Libelle_activite_etablissement_initial',
             'Numero_inspection_milieu2',
             'Numero_inspection_milieu3',
             'Numero_inspection_milieu_all',
             
             'Libelle_commune_initial',
             'SIRET',
             'Adresse_2_UA',
             'Adresse_2_UA_initial',
             'Code_postal',
             'Numero_inspection',
             'Date_inspection',
             'Synthese_eval_sanit',
             'Agrement',
             'geores',
             'y',
            ]

feats = [x for x in df if x not in forbidden and x not in useless and not(any(s in x for s in ['oof', '_y_', '_y']))]

print(len(feats))
df[feats].head()

892


,Libelle_commune,APP_Libelle_activite_etablissement,filtre,ods_type_activite,trancheEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,etablissementSiege,nombrePeriodesEtablissement,indiceRepetitionEtablissement,codeCommuneEtablissement,codeCedexEtablissement,activitePrincipaleEtablissement,caractereEmployeurEtablissement,codeCedexEtablissement_2char,codeCommuneEtablissement_2char,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,year_dateDebut,month_dateDebut,year_dateCreationEtablissement,month_dateCreationEtablissement,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_abattoir,tfidf_accueil,tfidf_agricole,tfidf_andre,tfidf_arc,tfidf_ass,tfidf_association,tfidf_atelier,tfidf_auberge,tfidf_auchan,tfidf_bar,tfidf_bat,tfidf_belle,tfidf_bio,tfidf_bistrot,tfidf_bois,tfidf_bon,tfidf_boucherie,tfidf_boulangerie,tfidf_brasserie,tfidf_burger,tfidf_cafe,tfidf_cantine,tfidf_carrefour,tfidf_centrale,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_chez,tfidf_chicken,tfidf_city,tfidf_clinique,tfidf_co,tfidf_college,tfidf_commune,tfidf_compagnie,tfidf_comptoir,tfidf_coop,tfidf_cooperative,tfidf_cote,tfidf_creche,tfidf_creperie,tfidf_cuisine,tfidf_dame,tfidf_delices,tfidf_di,tfidf_distribution,tfidf_domaine,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_elementaire,tfidf_enfance,tfidf_epicerie,tfidf_esat,tfidf_etablissements,tfidf_ets,tfidf_eurl,tfidf_exploitation,tfidf_express,tfidf_ferme,tfidf_fils,tfidf_fondation,tfidf_food,tfidf_fournil,tfidf_foyer,tfidf_france,tfidf_francois,tfidf_franprix,tfidf_freres,tfidf_fromagerie,tfidf_gaec,tfidf_gestion,tfidf_gourmand,tfidf_gourmet,tfidf_grand,tfidf_grill,tfidf_groupe,tfidf_hopital,tfidf_hospitalier,tfidf_hotel,tfidf_huitres,tfidf_intermarche,tfidf_jardins,tfidf_jean,tfidf_joseph,tfidf_kebab,tfidf_king,tfidf_leclerc,tfidf_louis,tfidf_lycee,tfidf_mairie,tfidf_maison,tfidf_marche,tfidf_maree,tfidf_marie,tfidf_market,tfidf_mas,tfidf_maternelle,tfidf_mer,tfidf_metro,tfidf_michel,tfidf_moulin,tfidf_nouvelle,tfidf_ogec,tfidf_pain,tfidf_palais,tfidf_parc,tfidf_paris,tfidf_paul,tfidf_pays,tfidf_petit,tfidf_petite,tfidf_petits,tfidf_philippe,tfidf_pierre,tfidf_pizza,tfidf_poissonnerie,tfidf_primaire,tfidf_professionnel,tfidf_publique,tfidf_relais,tfidf_residence,tfidf_restaurant,tfidf_restauration,tfidf_retraite,tfidf_royal,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_sas,tfidf_saveurs,tfidf_scea,tfidf_scolaire,tfidf_snack,tfidf_soc,tfidf_ste,tfidf_sud,tfidf_super,tfidf_supermarche,tfidf_sushi,tfidf_table,tfidf_tacos,tfidf_traiteur,tfidf_val,tfidf_viande,tfidf_viandes,tfidf_volailles,tfidf_wok,tfidf_adresse_avenue,tfidf_adresse_boulevard,tfidf_adresse_che,tfidf_adresse_chemin,tfidf_adresse_gaulle,tfidf_adresse_general,tfidf_adresse_jean,tfidf_adresse_nan,tfidf_adresse_pierre,tfidf_adresse_place,tfidf_adresse_quai,tfidf_adresse_route,tfidf_adresse_rue,tfidf_adresse_saint,tfidf_adresse_zone,Ville,Departement,flag_Abattage de produits de l'aquaculture,flag_Alimentation générale,flag_Autres activités de remise directe,flag_Boucherie-Charcuterie,flag_Boulangerie-Pâtisserie,flag_Caves d'affinage,flag_Centre d'emballage des oeufs,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Chaîne d'abattage de volaille/lagomorphe/petit gibier,flag_Chocolatier,flag_Collecte de gibier sauvage,flag_Collecte de lait,flag_Collecte et transformation de miel,flag_Collecteur d'oeufs,flag_Commerce alimentaire,flag_Distribution automatique,flag_Découpe de fromages,flag_Découpe de gibier sauvage,flag_Découpe de gros gibier d'élevage,flag_Découpe de produits de charcuterie,flag_Découpe de viande de boucherie,flag_Découpe de viandes de volailles/lagomorphe,flag_Déshydratation de lait ou produits laitiers,flag_Elevage de bovins,flag_Entreposage de distribution de denrées alimentaires,flag_Entreposage de produits de la pêche,flag_Entreposage de produits laitiers,flag_Entreposage de viande de boucherie,flag_Entreposage de viande de volailles,flag_Entreposa

In [9]:
feats = ['Libelle_commune',
 'APP_Libelle_activite_etablissement',
 'filtre',
 'ods_type_activite',
 'etablissementSiege',
 'codeCommuneEtablissement',
 'activitePrincipaleEtablissement',
 'codeCommuneEtablissement_2char',
 'Code_NAF_1',
 'Code_NAF_2',
 'Egalite_NAF',
 'tfidf_ass',
 'tfidf_atelier',
 'tfidf_bistrot',
 'tfidf_boucherie',
 'tfidf_centre',
 'tfidf_charcuterie',
 'tfidf_chateau',
 'tfidf_college',
 'tfidf_commune',
 'tfidf_delices',
 'tfidf_donald',
 'tfidf_earl',
 'tfidf_ecole',
 'tfidf_ehpad',
 'tfidf_etablissements',
 'tfidf_eurl',
 'tfidf_exploitation',
 'tfidf_fils',
 'tfidf_gaec',
 'tfidf_groupe',
 'tfidf_king',
 'tfidf_leclerc',
 'tfidf_mairie',
 'tfidf_maree',
 'tfidf_market',
 'tfidf_mas',
 'tfidf_pain',
 'tfidf_paris',
 'tfidf_saint',
 'tfidf_salaisons',
 'tfidf_sarl',
 'tfidf_saveurs',
 'tfidf_soc',
 'tfidf_sushi',
 'tfidf_viandes',
 'tfidf_adresse_pierre',
 'tfidf_adresse_saint',
 'Ville',
 'Departement',
 'flag_Autres activités de remise directe',
 'flag_Boulangerie-Pâtisserie',
 "flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage",
 'flag_Entreposage de distribution de denrées alimentaires',
 'flag_Métier de bouche',
 'flag_Producteur fermier',
 'flag_Rayon poissonnerie',
 'flag_Restauration collective',
 'flag_Transformation de lait ou produits laitiers',
 'flag_Transport de denrées alimentaires',
 'Agrement_num_commune',
 'Agrement_num_entreprise',
 'Numero_inspection_len',
 'Numero_inspection_debut',
 'Numero_inspection_fin',
 'year',
 'dayofyear',
 'is_restaurant',
 'is_vendeur',
 'is_rayon',
 'is_boucherie',
 'missing_agrement',
         
         
 'other_y0_AGREMENTS_in_dftrain',
 'other_y1_AGREMENTS_in_dftrain',
 'other_y2_AGREMENTS_in_dftrain',
 'other_y3_AGREMENTS_in_dftrain',  
         
 'ndays_since_last_control',
 'ndays_since_older_control',
 'ndays_before_next_control',
 'ndays_before_furthest_control',
 'result_last_control',
 'result_older_control',
 'result_next_control',
 'result_furthest_control',
 'N_agrements_in_train_other_day',
 'N_agrements_in_train_past',
 'N_agrements_in_train_future',
 'ndays_MEAN_since_past_controls',
 'ndays_STD_since_past_controls',
 'ndays_MEAN_before_futur_controls',
 'ndays_STD_before_futur_controls',
         
 'Diffdate_days_DateInspection_dateDebut',
 'Diffdate_days_DateInspection_dateCreationEtablissement',
             
 'N_agrements_in_train',      
 'y_last_1th_control',
 'latitude',
 'longitude',
 'Days_since_next_control_to_REAL_3th_nearest',
 'Days_since_next_control_to_REAL_6th_nearest',
 'Days_since_next_control_to_REAL_10th_nearest',
 'Days_since_next_control_to_REAL_15th_nearest',
 'Days_since_next_control_to_REAL_20th_nearest',
 'MAX_dist_to_REAL_50th_nearest',
 'Days_since_next_control_to_3th_nearest_LibAct',
 'Days_since_next_control_to_6th_nearest_LibAct',
 'Days_since_next_control_to_10th_nearest_filtre',
 'MEAN_dist_to_20th_nearest_filtre',
 'MAX_dist_to_20th_nearest_filtre',
 'Days_since_next_control_to_20th_nearest_filtre',
 'MEAN_dist_to_3th_nearest_ods',
 'Days_since_next_control_to_3th_nearest_ods',
 'MAX_dist_to_6th_nearest_ods',
 'Days_since_next_control_to_6th_nearest_ods',
 'MEAN_dist_to_10th_nearest_ods',
 'MAX_dist_to_10th_nearest_ods',
 'Days_since_next_control_to_10th_nearest_ods',
 'MEAN_dist_to_20th_nearest_ods',
 'MAX_dist_to_20th_nearest_ods',
 'Days_since_next_control_to_20th_nearest_ods',
 'Days_since_nearest_control_to_20th_nearest_ods',
 'Days_since_next_control_to_3th_nearest_NAF1',
 'Days_since_next_control_to_6th_nearest_NAF1',
 'Days_since_next_control_to_10th_nearest_NAF1',
 'MEAN_dist_to_20th_nearest_NAF1',
 'STD_dist_to_20th_nearest_NAF1',
 'MAX_dist_to_20th_nearest_NAF1',
 'Days_since_next_control_to_20th_nearest_NAF1',
 'Days_since_next_control_to_3th_nearest_actPrincipal',
 'Days_since_next_control_to_20th_nearest_actPrincipal',
 'N_targets_1_within_0.03km',
 'N_targets_2_within_0.03km',
 'N_targets_3_within_0.03km',
 'N_NAF1_3_within_0.03km',
 'N_controls_same_day_within_0.03km',
 'N_POI_within_0.05km',
 'N_targets_1_within_0.05km',
 'N_targets_2_within_0.05km',
 'N_targets_3_within_0.05km',
 'N_NAF1_0_within_0.05km',
 'N_targets_1_within_0.075km',
 'N_targets_2_within_0.075km',
 'N_targets_3_within_0.075km',
 'N_NAF1_0_within_0.075km',
 'N_targets_1_within_0.1km',
 'N_targets_2_within_0.1km',
 'N_targets_3_within_0.1km',
 'N_NAF1_0_within_0.1km',
 'N_NAF1_1_within_0.1km',
 'N_NAF1_2_within_0.1km',
 'N_NAF1_5_within_0.1km',
 'N_POI_within_0.15km',
 'N_targets_1_within_0.15km',
 'N_targets_2_within_0.15km',
 'N_targets_3_within_0.15km',
 'N_NAF1_0_within_0.15km',
 'N_NAF1_2_within_0.15km',
 'N_POI_within_0.3km',
 'MEAN_dist_POI_within_0.3km',
 'N_targets_1_within_0.3km',
 'N_targets_2_within_0.3km',
 'N_targets_3_within_0.3km',
 'N_NAF1_0_within_0.3km',
 'N_NAF1_1_within_0.3km',
 'N_NAF1_2_within_0.3km',
 'N_NAF1_3_within_0.3km',
 'N_NAF1_4_within_0.3km',
 'N_POI_within_0.6km',
 'STD_dist_POI_within_0.6km',
 'N_targets_1_within_0.6km',
 'N_targets_2_within_0.6km',
 'N_targets_3_within_0.6km',
 'N_NAF1_0_within_0.6km',
 'N_NAF1_1_within_0.6km',
 'N_NAF1_2_within_0.6km',
 'N_NAF1_3_within_0.6km',
 'N_NAF1_4_within_0.6km',
 'N_NAF1_5_within_0.6km',
 'N_NAF1_6_within_0.6km',
 'N_controls_within_100days_and_0.6km',
 'N_POI_within_1km',
 'MEAN_dist_POI_within_1km',
 'STD_dist_POI_within_1km',
 'N_targets_0_within_1km',
 'N_targets_1_within_1km',
 'N_targets_2_within_1km',
 'N_targets_3_within_1km',
 'N_NAF1_0_within_1km',
 'N_NAF1_1_within_1km',
 'N_NAF1_2_within_1km',
 'N_NAF1_3_within_1km',
 'N_NAF1_4_within_1km',
 'N_NAF1_5_within_1km',
 'N_NAF1_6_within_1km',
 'N_controls_within_100days_and_1km',
 'N_POI_within_1.5km',
 'MEAN_dist_POI_within_1.5km',
 'STD_dist_POI_within_1.5km',
 'N_targets_0_within_1.5km',
 'N_targets_1_within_1.5km',
 'N_targets_2_within_1.5km',
 'N_targets_3_within_1.5km',
 'N_NAF1_0_within_1.5km',
 'N_NAF1_1_within_1.5km',
 'N_NAF1_2_within_1.5km',
 'N_NAF1_3_within_1.5km',
 'N_NAF1_4_within_1.5km',
 'N_NAF1_5_within_1.5km',
 'N_NAF1_6_within_1.5km',
 'N_controls_within_100days_and_1.5km',
 'N_controls_within_100daysPOS_and_1.5km',
 'N_controls_within_100daysNEG_and_1.5km',
 'N_POI_within_2km',
 'MEAN_dist_POI_within_2km',
 'STD_dist_POI_within_2km',
 'N_targets_0_within_2km',
 'N_targets_1_within_2km',
 'N_targets_2_within_2km',
 'N_targets_3_within_2km',
 'N_NAF1_0_within_2km',
 'N_NAF1_1_within_2km',
 'N_NAF1_2_within_2km',
 'N_NAF1_3_within_2km',
 'N_NAF1_4_within_2km',
 'N_NAF1_5_within_2km',
 'N_NAF1_6_within_2km',
 'N_controls_within_31days_and_2km',
 'N_controls_within_61days_and_2km',
 'N_controls_within_61daysNEG_and_2km',
 'N_controls_within_100days_and_2km',
 'N_controls_within_100daysPOS_and_2km',
 'N_controls_within_100daysNEG_and_2km',
 'N_POI_within_4km',
 'MEAN_dist_POI_within_4km',
 'STD_dist_POI_within_4km',
 'N_targets_0_within_4km',
 'N_targets_1_within_4km',
 'N_targets_2_within_4km',
 'N_targets_3_within_4km',
 'N_NAF1_0_within_4km',
 'N_NAF1_1_within_4km',
 'N_NAF1_2_within_4km',
 'N_NAF1_3_within_4km',
 'N_NAF1_4_within_4km',
 'N_NAF1_5_within_4km',
 'N_NAF1_6_within_4km',
 'N_controls_within_31days_and_4km',
 'N_controls_within_31daysPOS_and_4km',
 'N_controls_within_61days_and_4km',
 'N_controls_within_61daysNEG_and_4km',
 'N_controls_within_100days_and_4km',
 'N_controls_within_100daysPOS_and_4km',
 'N_controls_within_100daysNEG_and_4km',
]

feats += [x for x in [
                        ]
          if x not in feats]

print(len(feats))
display(df[feats].head())

252


,Libelle_commune,APP_Libelle_activite_etablissement,filtre,ods_type_activite,etablissementSiege,codeCommuneEtablissement,activitePrincipaleEtablissement,codeCommuneEtablissement_2char,Code_NAF_1,Code_NAF_2,Egalite_NAF,tfidf_ass,tfidf_atelier,tfidf_bistrot,tfidf_boucherie,tfidf_centre,tfidf_charcuterie,tfidf_chateau,tfidf_college,tfidf_commune,tfidf_delices,tfidf_donald,tfidf_earl,tfidf_ecole,tfidf_ehpad,tfidf_etablissements,tfidf_eurl,tfidf_exploitation,tfidf_fils,tfidf_gaec,tfidf_groupe,tfidf_king,tfidf_leclerc,tfidf_mairie,tfidf_maree,tfidf_market,tfidf_mas,tfidf_pain,tfidf_paris,tfidf_saint,tfidf_salaisons,tfidf_sarl,tfidf_saveurs,tfidf_soc,tfidf_sushi,tfidf_viandes,tfidf_adresse_pierre,tfidf_adresse_saint,Ville,Departement,flag_Autres activités de remise directe,flag_Boulangerie-Pâtisserie,flag_Chaîne d'abattage d'animaux boucherie/Gibier ongulé élevage,flag_Entreposage de distribution de denrées alimentaires,flag_Métier de bouche,flag_Producteur fermier,flag_Rayon poissonnerie,flag_Restauration collective,flag_Transformation de lait ou produits laitiers,flag_Transport de denrées alimentaires,Agrement_num_commune,Agrement_num_entreprise,Numero_inspection_len,Numero_inspection_debut,Numero_inspection_fin,year,dayofyear,is_restaurant,is_vendeur,is_rayon,is_boucherie,missing_agrement,other_y0_AGREMENTS_in_dftrain,other_y1_AGREMENTS_in_dftrain,other_y2_AGREMENTS_in_dftrain,other_y3_AGREMENTS_in_dftrain,ndays_since_last_control,ndays_since_older_control,ndays_before_next_control,ndays_before_furthest_control,result_last_control,result_older_control,result_next_control,result_furthest_control,N_agrements_in_train_other_day,N_agrements_in_train_past,N_agrements_in_train_future,ndays_MEAN_since_past_controls,ndays_STD_since_past_controls,ndays_MEAN_before_futur_controls,ndays_STD_before_futur_controls,Diffdate_days_DateInspection_dateDebut,Diffdate_days_DateInspection_dateCreationEtablissement,N_agrements_in_train,y_last_1th_control,latitude,longitude,Days_since_next_control_to_REAL_3th_nearest,Days_since_next_control_to_REAL_6th_nearest,Days_since_next_control_to_REAL_10th_nearest,Days_since_next_control_to_REAL_15th_nearest,Days_since_next_control_to_REAL_20th_nearest,MAX_dist_to_REAL_50th_nearest,Days_since_next_control_to_3th_nearest_LibAct,Days_since_next_control_to_6th_nearest_LibAct,Days_since_next_control_to_10th_nearest_filtre,MEAN_dist_to_20th_nearest_filtre,MAX_dist_to_20th_nearest_filtre,Days_since_next_control_to_20th_nearest_filtre,MEAN_dist_to_3th_nearest_ods,Days_since_next_control_to_3th_nearest_ods,MAX_dist_to_6th_nearest_ods,Days_since_next_control_to_6th_nearest_ods,MEAN_dist_to_10th_nearest_ods,MAX_dist_to_10th_nearest_ods,Days_since_next_control_to_10th_nearest_ods,MEAN_dist_to_20th_nearest_ods,MAX_dist_to_20th_nearest_ods,Days_since_next_control_to_20th_nearest_ods,Days_since_nearest_control_to_20th_nearest_ods,Days_since_next_control_to_3th_nearest_NAF1,Days_since_next_control_to_6th_nearest_NAF1,Days_since_next_control_to_10th_nearest_NAF1,MEAN_dist_to_20th_nearest_NAF1,STD_dist_to_20th_nearest_NAF1,MAX_dist_to_20th_nearest_NAF1,Days_since_next_control_to_20th_nearest_NAF1,Days_since_next_control_to_3th_nearest_actPrincipal,Days_since_next_control_to_20th_nearest_actPrincipal,N_targets_1_within_0.03km,N_targets_2_within_0.03km,N_targets_3_within_0.03km,N_NAF1_3_within_0.03km,N_controls_same_day_within_0.03km,N_POI_within_0.05km,N_targets_1_within_0.05km,N_targets_2_within_0.05km,N_targets_3_within_0.05km,N_NAF1_0_within_0.05km,N_targets_1_within_0.075km,N_targets_2_within_0.075km,N_targets_3_within_0.075km,N_NAF1_0_within_0.075km,N_targets_1_within_0.1km,N_targets_2_within_0.1km,N_targets_3_within_0.1km,N_NAF1_0_within_0.1km,N_NAF1_1_within_0.1km,N_NAF1_2_within_0.1km,N_NAF1_5_within_0.1km,N_POI_within_0.15km,N_targets_1_within_0.15km,N_targets_2_within_0.15km,N_targets_3_within_0.15km,N_NAF1_0_within_0.15km,N_NAF1_2_within_0.15km,N_POI_within_0.3km,MEAN_dist_POI_within_0.3km,N_targets_1_within_0.3km,N_tar

In [10]:
nb_folds = 10
skf = StratifiedKFold(n_splits=nb_folds, shuffle=True, random_state=1)

for i, (train_index, test_index) in enumerate(skf.split(df.index, df['y'].astype(str))):
    print(i, df.loc[train_index].shape[0], df.loc[test_index].shape[0])

0 20403 2268
1 20404 2267
2 20404 2267
3 20404 2267
4 20404 2267
5 20404 2267
6 20404 2267
7 20404 2267
8 20404 2267
9 20404 2267


In [11]:
target_classes = list(dico_y.keys())
target_classes

['A corriger de manière urgente',
 'A améliorer',
 'Satisfaisant',
 'Très satisfaisant']

In [13]:
initial_columns_df = list(df.columns)
initial_columns_test_df = list(test_df.columns)

___
___
# **XGBOOST**

In [14]:
model_params_1 = {
                    "objective": "multi:softprob",
                    'eval_metric':'mlogloss',
                    "random_state": 42,

                    'learning_rate': 0.04,
                    'n_estimators': 5000,
                    'max_depth': 9,
                    'early_stopping_rounds': 50,
                    'gamma': 0,
                    'min_child_weight': 1,
                    'subsample': 0.9,
                    'colsample_bytree': 0.4,
                    'nthread' : -1,
                    'verbosity':0,
    
                    # GPU
                    #'tree_method':"hist",
                    'tree_method':'gpu_hist',
                    'gpu_id': 0, # Or None if cpu
                }

model_params_2 = {
                    "objective": "multi:softprob",
                    'eval_metric':'mlogloss',
                    "random_state": 42,

                    'learning_rate': 0.045,
                    'n_estimators': 5000,
                    'max_depth': 9,
                    'early_stopping_rounds': 40,
                    'gamma': 0.2,
                    'reg_alpha' : 1,
                    'min_child_weight': 0.5,
                    'subsample': 0.8,
                    'colsample_bytree': 0.35,
                    'nthread' : -1,
                    'verbosity':0,
    
                    # GPU
                    #'tree_method':"hist",
                    'tree_method':'gpu_hist',
                    'gpu_id': 0, # Or None if cpu
                }

model_params_3 = {
                    "objective": "multi:softprob",
                    'eval_metric':'mlogloss',
                    "random_state": 42,

                    'learning_rate': 0.04,
                    'n_estimators': 5000,
                    'max_depth': 8,
                    'early_stopping_rounds': 40,
                    'gamma': 0.3,
                    'reg_alpha' : 1,
                    'min_child_weight': 0.3,
                    'subsample': 0.8,
                    'colsample_bytree': 0.45,
                    'nthread' : -1,
                    'verbosity':0,
    
                    # GPU
                    #'tree_method':"hist",
                    'tree_method':'gpu_hist',
                    'gpu_id': 0, # Or None if cpu
                }

model_params_4 = {
                    "objective": "multi:softprob",
                    'eval_metric':'mlogloss',
                    "random_state": 42,

                    'learning_rate': 0.025,
                    'n_estimators': 5000,
                    'max_depth': 11,
                    'early_stopping_rounds': 30,
                    'gamma': 0.3,
                    'reg_alpha' : 0.5,
                    'min_child_weight': 0.2,
                    'subsample': 0.8,
                    'colsample_bytree': 0.42,
                    'nthread' : -1,
                    'verbosity':0,
    
                    # GPU
                    #'tree_method':"hist",
                    'tree_method':'gpu_hist',
                    'gpu_id': 0, # Or None if cpu
                }

model_params_5 = {
                    "objective": "multi:softprob",
                    'eval_metric':'mlogloss',
                    "random_state": 42,

                    'learning_rate': 0.032,
                    'n_estimators': 5000,
                    'max_depth': 10,
                    'early_stopping_rounds': 50,
                    'gamma': 0.05,
                    'min_child_weight': 1.2,
                    'subsample': 0.94,
                    'colsample_bytree': 0.38,
                    'nthread' : -1,
                    'verbosity':0,
    
                    # GPU
                    #'tree_method':"hist",
                    'tree_method':'gpu_hist',
                    'gpu_id': 0, # Or None if cpu
                }

model_params_6 = {
                    "objective": "multi:softprob",
                    'eval_metric':'mlogloss',
                    "random_state": 42,

                    'learning_rate': 0.045,
                    'n_estimators': 5000,
                    'max_depth': 9,
                    'early_stopping_rounds': 50,
                    'gamma': 0,
                    'reg_alpha' : 1,
                    'min_child_weight': 0.5,
                    'subsample': 0.95,
                    'colsample_bytree': 0.35,
                    'nthread' : -1,
                    'verbosity':0,
    
                    # GPU
                    #'tree_method':"hist",
                    'tree_method':'gpu_hist',
                    'gpu_id': 0, # Or None if cpu
                }

model_params_7 = {
                    "objective": "multi:softprob",
                    'eval_metric':'mlogloss',
                    "random_state": 42,

                    'learning_rate': 0.06,
                    'n_estimators': 5000,
                    'max_depth': 9,
                    'early_stopping_rounds': 50,
                    'gamma': 0.15,
                    'reg_alpha' : 1,
                    'min_child_weight': 0.7,
                    'subsample': 0.92,
                    'colsample_bytree': 0.46,
                    'nthread' : -1,
                    'verbosity':0,
    
                    # GPU
                    #'tree_method':"hist",
                    'tree_method':'gpu_hist',
                    'gpu_id': 0, # Or None if cpu
                }

model_params_8 = {
                    "objective": "multi:softprob",
                    'eval_metric':'mlogloss',
                    "random_state": 42,

                    'learning_rate': 0.035,
                    'n_estimators': 5000,
                    'max_depth': 11,
                    'early_stopping_rounds': 50,
                    'gamma': 0.25,
                    'reg_alpha' : 0.5,
                    'min_child_weight': 0.1,
                    'subsample': 0.9,
                    'colsample_bytree': 0.42,
                    'nthread' : -1,
                    'verbosity':0,
    
                    # GPU
                    #'tree_method':"hist",
                    'tree_method':'gpu_hist',
                    'gpu_id': 0, # Or None if cpu
                }


MODEL_PARAMS = [model_params_1,
                model_params_2,
                model_params_3,
                model_params_4,
                model_params_5,
                model_params_6,
                model_params_7,
                model_params_8,
               ]

if debug :
    MODEL_PARAMS = [model_params_1,
                    model_params_2,
                   ]

In [15]:
%%time

for num_params, clf_params in enumerate(MODEL_PARAMS):
    print()
    print('='*50)
    
    df[[f'oof_xgb_param{num_params}_{i}' for i in target_classes]] = 0
    df[f"oof_class_xgb_param{num_params}"] = None
    test_df[[f'pred_xgb_param{num_params}_{i}' for i in target_classes]] = 0    
    
    for i, (train_index, test_index) in enumerate(skf.split(df.index, df['y'].astype(str))):
        #print(f"KFold {i+1}/{skf.n_splits}.")

        # Création des ensembles d'apprentissage
        xtr, ytr = df.loc[train_index, feats], df.loc[train_index, 'y']
        xte, yte = df.loc[test_index, feats], df.loc[test_index, 'y']

        # Create model            
        model = xgb.XGBClassifier(**clf_params)

        # Fit model
        model.fit(xtr, ytr,
                  eval_set=[(xte, yte)],
                  verbose=False)

        # Store oof
        df.loc[test_index, [f'oof_xgb_param{num_params}_{dico_y_INVERSE[i]}' for i in model.classes_]] = model.predict_proba(xte)
        df.loc[test_index, f"oof_class_xgb_param{num_params}"] = model.predict(xte)
  
        # Test predictions
        test_df.loc[:, [f'pred_xgb_param{num_params}_{dico_y_INVERSE[i]}' for i in model.classes_]] += model.predict_proba(test_df[feats]) / skf.n_splits

        # Print score
        print(f"Score fold {i+1} : {round(accuracy_score(df.loc[test_index, 'y'], df.loc[test_index, f'oof_class_xgb_param{num_params}'].astype(int)), 5)}")

    # Print Score
    print(f"\nFinal score : {round(accuracy_score(df['y'], df[f'oof_class_xgb_param{num_params}'].astype(int)), 5)}")


Score fold 1 : 0.70018
Score fold 2 : 0.70843
Score fold 3 : 0.7071
Score fold 4 : 0.73577
Score fold 5 : 0.71195
Score fold 6 : 0.7049
Score fold 7 : 0.71504
Score fold 8 : 0.70666
Score fold 9 : 0.70931
Score fold 10 : 0.7146

Final score : 0.71139

Score fold 1 : 0.69797
Score fold 2 : 0.70225
Score fold 3 : 0.70666
Score fold 4 : 0.73136
Score fold 5 : 0.71416
Score fold 6 : 0.70446
Score fold 7 : 0.71151
Score fold 8 : 0.70975
Score fold 9 : 0.70798
Score fold 10 : 0.71681

Final score : 0.71029

Score fold 1 : 0.69753
Score fold 2 : 0.70181
Score fold 3 : 0.70754
Score fold 4 : 0.73136
Score fold 5 : 0.71725
Score fold 6 : 0.69872
Score fold 7 : 0.70578
Score fold 8 : 0.71151
Score fold 9 : 0.71151
Score fold 10 : 0.71195

Final score : 0.7095

Score fold 1 : 0.70062
Score fold 2 : 0.70181
Score fold 3 : 0.70798
Score fold 4 : 0.73357
Score fold 5 : 0.71637
Score fold 6 : 0.70578
Score fold 7 : 0.71328
Score fold 8 : 0.71504
Score fold 9 : 0.71284
Score fold 10 : 0.71769

Final 

___
___
# **Catboost**

In [18]:
categorical_feats = ['APP_Libelle_activite_etablissement',
                     'Libelle_commune',
                     'ods_type_activite',
                     'Ville',
                     'Departement',
                     'filtre',
                     'trancheEffectifsEtablissement',
                     'activitePrincipaleRegistreMetiersEtablissement',
                     'etablissementSiege',
                     'activitePrincipaleEtablissement',
                     'caractereEmployeurEtablissement',
                     'Code_NAF_1',
                     'Code_NAF_2',
                     'Egalite_NAF',
                    ] + [x for x in feats if x.startswith('Numero_inspection')]

df[categorical_feats]      = df[categorical_feats].astype(str)
test_df[categorical_feats] = test_df[categorical_feats].astype(str)

categorical_feats = [x for x in categorical_feats if x in feats]
categorical_feats

['APP_Libelle_activite_etablissement',
 'Libelle_commune',
 'ods_type_activite',
 'Ville',
 'Departement',
 'filtre',
 'etablissementSiege',
 'activitePrincipaleEtablissement',
 'Code_NAF_1',
 'Code_NAF_2',
 'Egalite_NAF',
 'Numero_inspection_len',
 'Numero_inspection_debut',
 'Numero_inspection_fin']

In [19]:
model_params_1 = {'learning_rate': 0.04,
                  'rsm': 1,
                  'l2_leaf_reg': 1,
                  'depth': 6,
                  'iterations': 3000,
                   #'subsample': 0.95,
                   #MultiLogloss
                  'cat_features' : categorical_feats,
                  
                    # GPU
                   'task_type' : "GPU",
                   'devices' : '0:1',
                   'bootstrap_type' : 'Poisson', # gaussian on CPU, Poisson on GPU

                    # Overfitting detector
                    'od_type' : "Iter",
                    'od_wait' : 30,
                  }

model_params_2 = {'learning_rate': 0.04,
                   'rsm': 1,
                   'l2_leaf_reg': 1,
                   'depth': 7,
                   'iterations': 5000,
                   #'subsample': 0.95,
                   #MultiLogloss
                   'cat_features' : categorical_feats,
                  
                    # GPU
                    'task_type' : "GPU",
                    'devices' : '0:1',
                    'bootstrap_type' : 'Poisson', # gaussian on CPU, Poisson on GPU

                    # Overfitting detector
                    'od_type' : "Iter",
                    'od_wait' : 30,
                  }

model_params_3 = {'learning_rate': 0.03,
                  'rsm': 1,
                  'l2_leaf_reg': 1,
                  'depth': 8,
                  'iterations': 4000,
                  'subsample': 0.95,
                   #MultiLogloss
                  'cat_features' : categorical_feats,
                  
                    # GPU
                   'task_type' : "GPU",
                   'devices' : '0:1',
                   'bootstrap_type' : 'Poisson', # gaussian on CPU, Poisson on GPU

                    # Overfitting detector
                    'od_type' : "Iter",
                    'od_wait' : 30,
                  }

model_params_4 = {'learning_rate': 0.03,
                  #'rsm': 0.5, # NO RSM  on GPU (colsample_bylevel)
                  'l2_leaf_reg': 3, # default value = 3
                  'depth': 6,
                  'iterations': 6000,
                   #'subsample': 0.95,
                   #MultiLogloss
                  'cat_features' : categorical_feats,
                  
                    # GPU
                   'task_type' : "GPU",
                   'devices' : '0:1',
                   'bootstrap_type' : 'Poisson', # gaussian on CPU, Poisson on GPU

                    # Overfitting detector
                    'od_type' : "Iter",
                    'od_wait' : 50,
                  }

model_params_5 = {'learning_rate': 0.035,
                  #'rsm': 0.5, # NO RSM  on GPU (colsample_bylevel)
                   'l2_leaf_reg': 0.5, # default value = 3
                   'depth': 7,
                   'iterations': 10000,
                   #'subsample': 0.95,
                   #MultiLogloss
                   'cat_features' : categorical_feats,
                  
                    # GPU
                    'task_type' : "GPU",
                    'devices' : '0:1',
                    'bootstrap_type' : 'Poisson', # gaussian on CPU, Poisson on GPU

                    # Overfitting detector
                    'od_type' : "Iter",
                    'od_wait' : 50,
                  }

MODEL_PARAMS = [model_params_1, model_params_2, model_params_3, model_params_4, model_params_5]

if debug :
    MODEL_PARAMS = [model_params_1,
                   ]

In [20]:
%%time

for num_params, clf_params in enumerate(MODEL_PARAMS):
    print()
    print('='*50)

    df[[f'oof_catboost_param{num_params}_{i}' for i in target_classes]] = 0
    df[f"oof_class_catboost_param{num_params}"] = None
    test_df[[f'pred_catboost_param{num_params}_{i}' for i in target_classes]] = 0

    for i, (train_index, test_index) in enumerate(skf.split(df.index, df['y'].astype(str))):
        #print(f"KFold {i+1}/{skf.n_splits}.")

        # Création des ensembles d'apprentissage
        xtr, ytr = df.loc[train_index, feats], df.loc[train_index, 'y']
        xte, yte = df.loc[test_index, feats], df.loc[test_index, 'y']

        model = catboost.CatBoostClassifier(**clf_params)

        # Fit model
        model.fit(xtr, ytr, verbose=False)

        # Store oof
        df.loc[test_index, [f'oof_catboost_param{num_params}_{dico_y_INVERSE[i]}' for i in model.classes_]] = model.predict_proba(xte)
        df.loc[test_index, f"oof_class_catboost_param{num_params}"] = model.predict(xte)
  
        # Test predictions
        test_df.loc[:, [f'pred_catboost_param{num_params}_{dico_y_INVERSE[i]}' for i in model.classes_]] += model.predict_proba(test_df[feats]) / skf.n_splits

        # Print score
        print(f"Score fold {i+1} : {round(accuracy_score(df.loc[test_index, 'y'], df.loc[test_index, f'oof_class_catboost_param{num_params}'].astype(int)), 5)}")

    # Print AUC
    print(f"\nFinal score : {round(accuracy_score(df['y'], df[f'oof_class_catboost_param{num_params}'].astype(int)), 5)}")


Score fold 1 : 0.69709
Score fold 2 : 0.7049
Score fold 3 : 0.70446
Score fold 4 : 0.72254
Score fold 5 : 0.70975
Score fold 6 : 0.70049
Score fold 7 : 0.70666
Score fold 8 : 0.70181
Score fold 9 : 0.70446
Score fold 10 : 0.70137

Final score : 0.70535

Score fold 1 : 0.69268
Score fold 2 : 0.70666
Score fold 3 : 0.70446
Score fold 4 : 0.72254
Score fold 5 : 0.70975
Score fold 6 : 0.70401
Score fold 7 : 0.70975
Score fold 8 : 0.7071
Score fold 9 : 0.70357
Score fold 10 : 0.70622

Final score : 0.70667

Score fold 1 : 0.69709
Score fold 2 : 0.70887
Score fold 3 : 0.71063
Score fold 4 : 0.7296
Score fold 5 : 0.71151
Score fold 6 : 0.70401
Score fold 7 : 0.7146
Score fold 8 : 0.71151
Score fold 9 : 0.71328
Score fold 10 : 0.71284

Final score : 0.71139

Score fold 1 : 0.69621
Score fold 2 : 0.70622
Score fold 3 : 0.70181
Score fold 4 : 0.72431
Score fold 5 : 0.7146
Score fold 6 : 0.69872
Score fold 7 : 0.70666
Score fold 8 : 0.70754
Score fold 9 : 0.70887
Score fold 10 : 0.7049

Final sc

___
___
# **Export**

In [23]:
cols = ['Numero_inspection'] + [x for x in df if x not in initial_columns_df]
df[cols].to_csv('train_oof_xgb_catboost.csv', index=False)
df[cols].head(1)

,Numero_inspection,oof_xgb_param0_A corriger de manière urgente,oof_xgb_param0_A améliorer,oof_xgb_param0_Satisfaisant,oof_xgb_param0_Très satisfaisant,oof_class_xgb_param0,oof_xgb_param1_A corriger de manière urgente,oof_xgb_param1_A améliorer,oof_xgb_param1_Satisfaisant,oof_xgb_param1_Très satisfaisant,oof_class_xgb_param1,oof_xgb_param2_A corriger de manière urgente,oof_xgb_param2_A améliorer,oof_xgb_param2_Satisfaisant,oof_xgb_param2_Très satisfaisant,oof_class_xgb_param2,oof_xgb_param3_A corriger de manière urgente,oof_xgb_param3_A améliorer,oof_xgb_param3_Satisfaisant,oof_xgb_param3_Très satisfaisant,oof_class_xgb_param3,oof_xgb_param4_A corriger de manière urgente,oof_xgb_param4_A améliorer,oof_xgb_param4_Satisfaisant,oof_xgb_param4_Très satisfaisant,oof_class_xgb_param4,oof_xgb_param5_A corriger de manière urgente,oof_xgb_param5_A améliorer,oof_xgb_param5_Satisfaisant,oof_xgb_param5_Très satisfaisant,oof_class_xgb_param5,oof_xgb_param6_A corriger de manière urgente,oof_xgb_param6_A améliorer,oof_xgb_param6_Satisfaisant,oof_xgb_param6_Très satisfaisant,oof_class_xgb_param6,oof_xgb_param7_A corriger de manière urgente,oof_xgb_param7_A améliorer,oof_xgb_param7_Satisfaisant,oof_xgb_param7_Très satisfaisant,oof_class_xgb_param7,oof_catboost_param0_A corriger de manière urgente,oof_catboost_param0_A améliorer,oof_catboost_param0_Satisfaisant,oof_catboost_param0_Très satisfaisant,oof_class_catboost_param0,oof_catboost_param1_A corriger de manière urgente,oof_catboost_param1_A améliorer,oof_catboost_param1_Satisfaisant,oof_catboost_param1_Très satisfaisant,oof_class_catboost_param1,oof_catboost_param2_A corriger de manière urgente,oof_catboost_param2_A améliorer,oof_catboost_param2_Satisfaisant,oof_catboost_param2_Très satisfaisant,oof_class_catboost_param2,oof_catboost_param3_A corriger de manière urgente,oof_catboost_param3_A améliorer,oof_catboost_param3_Satisfaisant,oof_catboost_param3_Très satisfaisant,oof_class_catboost_param3,oof_catboost_param4_A corriger de manière urgente,oof_catboost_param4_A améliorer,oof_catboost_param4_Satisfaisant,oof_catboost_param4_Très satisfaisant,oof_class_catboost_param4
0,23-110093-1,0.000261,0.0015,0.991794,0.006445,2,0.000785,0.00404,0.983477,0.011698,2,0.000852,0.004473,0.980456,0.014218,2,0.001181,0.004229,0.982526,0.012064,2,0.000631,0.002027,0.987796,0.009547,2,0.000782,0.004224,0.984618,0.010376,2,0.000463,0.002463,0.989539,0.007535,2,0.001009,0.002794,0.98636,0.009837,2,0.000277,0.027237,0.965033,0.007453,2,0.000021,0.006675,0.990377,0.002927,2,0.000047,0.006507,0.990411,0.003034,2,0.000204,0.018199,0.975697,0.0059,2,0.000001,0.001656,0.997762,0.000581,2


In [24]:
cols = ['Numero_inspection', 'trustii_id'] + [x for x in test_df if x not in initial_columns_test_df]
test_df[cols].to_csv('test_oof_xgb_catboost.csv', index=False)
test_df[cols].head(1)

,Numero_inspection,trustii_id,pred_xgb_param0_A corriger de manière urgente,pred_xgb_param0_A améliorer,pred_xgb_param0_Satisfaisant,pred_xgb_param0_Très satisfaisant,pred_xgb_param1_A corriger de manière urgente,pred_xgb_param1_A améliorer,pred_xgb_param1_Satisfaisant,pred_xgb_param1_Très satisfaisant,pred_xgb_param2_A corriger de manière urgente,pred_xgb_param2_A améliorer,pred_xgb_param2_Satisfaisant,pred_xgb_param2_Très satisfaisant,pred_xgb_param3_A corriger de manière urgente,pred_xgb_param3_A améliorer,pred_xgb_param3_Satisfaisant,pred_xgb_param3_Très satisfaisant,pred_xgb_param4_A corriger de manière urgente,pred_xgb_param4_A améliorer,pred_xgb_param4_Satisfaisant,pred_xgb_param4_Très satisfaisant,pred_xgb_param5_A corriger de manière urgente,pred_xgb_param5_A améliorer,pred_xgb_param5_Satisfaisant,pred_xgb_param5_Très satisfaisant,pred_xgb_param6_A corriger de manière urgente,pred_xgb_param6_A améliorer,pred_xgb_param6_Satisfaisant,pred_xgb_param6_Très satisfaisant,pred_xgb_param7_A corriger de manière urgente,pred_xgb_param7_A améliorer,pred_xgb_param7_Satisfaisant,pred_xgb_param7_Très satisfaisant,pred_xgb_final_A corriger de manière urgente,pred_xgb_final_A améliorer,pred_xgb_final_Satisfaisant,pred_xgb_final_Très satisfaisant,pred_catboost_param0_A corriger de manière urgente,pred_catboost_param0_A améliorer,pred_catboost_param0_Satisfaisant,pred_catboost_param0_Très satisfaisant,pred_catboost_param1_A corriger de manière urgente,pred_catboost_param1_A améliorer,pred_catboost_param1_Satisfaisant,pred_catboost_param1_Très satisfaisant,pred_catboost_param2_A corriger de manière urgente,pred_catboost_param2_A améliorer,pred_catboost_param2_Satisfaisant,pred_catboost_param2_Très satisfaisant,pred_catboost_param3_A corriger de manière urgente,pred_catboost_param3_A améliorer,pred_catboost_param3_Satisfaisant,pred_catboost_param3_Très satisfaisant,pred_catboost_param4_A corriger de manière urgente,pred_catboost_param4_A améliorer,pred_catboost_param4_Satisfaisant,pred_catboost_param4_Très satisfaisant,pred_catboost_final_A corriger de manière urgente,pred_catboost_final_A améliorer,pred_catboost_final_Satisfaisant,pred_catboost_final_Très satisfaisant
0,23-003935-1,1.0,0.001476,0.060738,0.47114,0.466646,0.00214,0.056152,0.467905,0.473803,0.0017,0.058279,0.493188,0.446833,0.004206,0.051029,0.486639,0.458127,0.002149,0.064035,0.486535,0.447281,0.001764,0.057081,0.493627,0.447528,0.001731,0.06629,0.469478,0.462501,0.003984,0.054629,0.474375,0.467013,0.002394,0.058529,0.480361,0.458717,0.000327,0.069373,0.469097,0.461202,0.000085,0.046059,0.49832,0.455536,0.000191,0.048287,0.53102,0.420503,0.000446,0.072923,0.44023,0.486401,0.000015,0.021002,0.515743,0.463239,0.000213,0.051529,0.490882,0.457376
